In [12]:
# Class 모델 이용해서 구현

sentences = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

import re

# remove special characters and replace with ' '
# [] => 나열된 문자 혹은 범위에 해당하는 문자

# 특수문자 제거
sentences = re.sub('[^A-Za-z0-9]+', ' ', sentences)

# remove 1 letter word 첫부분 한단어,끝부분 한단어, 중간 한단어짜리 지움
# ?: non-capturing group -> 대충 매칭되도 무시하겠다 이런뜻
sentences =  re.sub(r'(?:^| )\w(?:$| )', ' ', sentences).strip()

# lower all characters
sentences = sentences.lower()

In [13]:
words = sentences.split()
vocab = set(words)
vocab_size = len(vocab)
embed_dim = 10

# sliding window 의 한쪽 면이라고 생각하는게 편한거 같다. 
context_size = 2

# implementation
word_to_idx = {word: i for i,word in enumerate(vocab)}
idx_to_word = {i:word for i,word in enumerate(vocab)}

data = []
for center_word_pos in range(len(words)):
    center_word_idx = word_to_idx[words[center_word_pos]]
    for w in range(-context_size, context_size+1):
        context_word_pos = center_word_pos + w
        if context_word_pos < 0 or context_word_pos >= len(words) or center_word_pos == context_word_pos:
            continue
        context_word_idx = word_to_idx[words[context_word_pos]]
        data.append((center_word_idx, context_word_idx))
print(len(data))
# 변수 정리 
# vocab (43개 set) : 문장 단어set 변환
# context_size : window 크기
# word_to_idx : 단어보고 숫자 변환
# idx_to_word : 숫자보고 단어 변환
# data : [훈련시킬 데이터, 정답라벨] 리스트

226


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define the model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        # input dimension 은 2차원이 되어야함
        self.layer_1 = nn.Linear(43, 10)
        self.layer_2 = nn.Linear(10, 43)
    def forward(self, x):
        x = self.layer_1(x)
        x = self.layer_2(x)
        return x

model = MyModel()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
num_epochs = 100

for epoch in range(num_epochs):
    loss_val = 0
    for context, target in data:
        
        input_one_hot = torch.zeros(len(word_to_idx))
        input_one_hot[context] = 1
        input_one_hot.unsqueeze_(0)

        # label 차원은 (1,1) 이 아닌 (1, ) 이 되어야한다.
        # (1,) =>
        label = torch.Tensor([target]).long()
        
        optimizer.zero_grad()
        outputs = model(input_one_hot)
        loss = criterion(outputs,label)
        loss_val += loss.item()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        print(f'Loss at epoch {epoch} : {loss_val/len(data)}')   

# 정확도 측정은 어떻게 하지?

Loss at epoch 0 : 3.862834119163783
Loss at epoch 10 : 2.171995179315584
Loss at epoch 20 : 2.0106027065652663
Loss at epoch 30 : 1.9727767426355751
Loss at epoch 40 : 1.9562668863650972
Loss at epoch 50 : 1.9466003981311764
Loss at epoch 60 : 1.9399442501300204
Loss at epoch 70 : 1.9348694716407135
Loss at epoch 80 : 1.931030275547399
Loss at epoch 90 : 1.9291293549854143
